pip install RPi.GPIO
pip install gspread
pip install oauth2client


In [ ]:
import os
import sys
import RPi.GPIO as GPIO
import serial
import threading
import datetime
import time
import logging
import traceback
import re
import tkinter as tk
from tkinter import ttk
from tkinter import filedialog  # For directory selection
import gspread
from google.oauth2.service_account import Credentials
import queue
import csv

# Logging configuration
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s [%(levelname)s] %(message)s',
    stream=sys.stdout
)

# GPIO setup for TTL mode
GPIO.setmode(GPIO.BCM)
GPIO.setwarnings(False)

# Define GPIO pins per device in TTL mode
gpio_pins_per_device = {
    'Port 1': {"LeftPoke": 17, "RightPoke": 27, "Pellet": 22},
    'Port 2': {"LeftPoke": 10, "RightPoke": 9, "Pellet": 11},
    'Port 3': {"LeftPoke": 0, "RightPoke": 5, "Pellet": 6},
    'Port 4': {"LeftPoke": 13, "RightPoke": 19, "Pellet": 26},
}

# Global event to signal threads to stop
stop_event = threading.Event()

# Define column headers for CSV and Google Sheets logging
column_headers = [
    "Timestamp", "Temp", "Humidity", "Library_Version", "Session_type",
    "Device_Number", "Battery_Voltage", "Motor_Turns", "FR", "Event", "Active_Poke",
    "Left_Poke_Count", "Right_Poke_Count", "Pellet_Count", "Block_Pellet_Count",
    "Retrieval_Time", "InterPelletInterval", "Poke_Time"
]

# Google Sheets credentials
SCOPE = ["https://spreadsheets.google.com/feeds", 'https://www.googleapis.com/auth/spreadsheets',
         "https://www.googleapis.com/auth/drive.file", "https://www.googleapis.com/auth/drive"]

class FED3MonitorApp:

    def __init__(self, root):
        self.root = root
        self.root.title("HPFED DATA MONITOR V.02")
        
        self.port_widgets = {}
        self.port_queues = {}
        self.experimenter_name = tk.StringVar()
        self.experiment_name = tk.StringVar()
        self.save_path = ""
        self.json_file_path = tk.StringVar()
        self.spreadsheet_id = tk.StringVar()
        self.mode = tk.StringVar(value="TTL")

        # To store data during the session
        self.data_to_save = {}

        # Set up the mainframe
        self.mainframe = ttk.Frame(self.root, padding="3 3 12 12")
        self.mainframe.grid(column=0, row=0, sticky=(tk.N, tk.W, tk.E, tk.S))
        self.mainframe.columnconfigure(0, weight=1)
        self.mainframe.rowconfigure(0, weight=1)
        
        # Add input fields for JSON file, Spreadsheet ID, and Data folder
        self.create_inputs()

        # Canvas for recording indicator (used for TTL mode)
        self.canvas = tk.Canvas(self.mainframe, width=200, height=200)
        self.canvas.grid(column=1, row=6, columnspan=2, padx=5, pady=5)
        self.recording_circle = None
        self.recording_label = None

        # Check for connected devices immediately
        self.check_connected_devices()

        # Start the periodic GUI update function
        self.root.after(100, self.update_gui)

    def create_inputs(self):
        # Experimenter name
        tk.Label(self.mainframe, text="Your Name:", font=("Helvetica", 10, "bold")).grid(column=0, row=2, sticky=tk.E, padx=2, pady=2)
        self.experimenter_entry = ttk.Entry(self.mainframe, textvariable=self.experimenter_name)
        self.experimenter_entry.grid(column=1, row=2, sticky=tk.W, padx=2, pady=2)

        # Experiment name
        tk.Label(self.mainframe, text="Experiment Name:", font=("Helvetica", 10, "bold")).grid(column=2, row=2, sticky=tk.E, padx=2, pady=2)
        self.experiment_entry = ttk.Entry(self.mainframe, textvariable=self.experiment_name)
        self.experiment_entry.grid(column=3, row=2, sticky=tk.W, padx=2, pady=2)

        # Mode selection
        tk.Label(self.mainframe, text="Mode:", font=("Helvetica", 10, "bold")).grid(column=0, row=3, sticky=tk.E, padx=2, pady=2)
        self.mode_selector = ttk.Combobox(self.mainframe, textvariable=self.mode, values=["TTL", "RTFED"])
        self.mode_selector.grid(column=1, row=3, sticky=tk.W, padx=2, pady=2)

        # JSON file path
        tk.Label(self.mainframe, text="JSON File Path:", font=("Helvetica", 10, "bold")).grid(column=2, row=3, sticky=tk.E, padx=2, pady=2)
        self.json_entry = ttk.Entry(self.mainframe, textvariable=self.json_file_path)
        self.json_entry.grid(column=3, row=3, sticky=tk.W, padx=2, pady=2)

        # Browse button for JSON file
        self.browse_json_button = tk.Button(self.mainframe, text="Browse JSON File", font=("Helvetica", 10, "bold"), command=self.browse_json_file, bg="lightblue")
        self.browse_json_button.grid(column=4, row=3, padx=5, pady=2)

        # Spreadsheet ID
        tk.Label(self.mainframe, text="Spreadsheet ID:", font=("Helvetica", 10, "bold")).grid(column=0, row=4, sticky=tk.E, padx=2, pady=2)
        self.spreadsheet_entry = ttk.Entry(self.mainframe, textvariable=self.spreadsheet_id)
        self.spreadsheet_entry.grid(column=1, row=4, sticky=tk.W, padx=2, pady=2)

        # Browse button for selecting data folder path (TTL mode only)
        self.browse_button = tk.Button(self.mainframe, text="Browse Data Folder", font=("Helvetica", 15, "bold"), command=self.browse_folder, bg="gold", fg="blue")
        self.browse_button.grid(column=2, row=4, padx=5, pady=10)

        # Start button
        self.start_button = tk.Button(self.mainframe, text="START", font=("Helvetica", 15, "bold"), bg="green", fg="white", command=self.start_experiment)
        self.start_button.grid(column=1, row=5, padx=5, pady=10)

        # Stop button
        self.stop_button = tk.Button(self.mainframe, text="STOP(SAVE & QUIT)", font=("Helvetica", 15, "bold"), bg="red", fg="white", command=self.stop_experiment)
        self.stop_button.grid(column=2, row=5, padx=5, pady=10)

    def browse_json_file(self):
        # Open file dialog to select JSON file
        json_file = filedialog.askopenfilename(title="Select JSON File", filetypes=[("JSON Files", "*.json")])
        if json_file:
            self.json_file_path.set(json_file)

    def browse_folder(self):
        # Allow the user to select a directory to save files (TTL mode only)
        self.save_path = filedialog.askdirectory(title="Select Folder to Save Data")

    def check_connected_devices(self):
        # Dynamically check and create port widgets based on connected devices
        self.device_mappings = self.get_device_mappings_by_usb_port()

        for idx, mapping in enumerate(self.device_mappings):
            port_identifier = mapping['port_identifier']
            frame = ttk.LabelFrame(self.mainframe, text=port_identifier)
            frame.grid(column=idx, row=0, padx=5, pady=5, sticky=(tk.N, tk.S, tk.E, tk.W))
            status_label = ttk.Label(frame, text="Not Ready", font=("Helvetica", 14, "italic"), foreground="red")
            status_label.grid(column=0, row=0, sticky=tk.W)
            text_widget = tk.Text(frame, width=40, height=20)
            text_widget.grid(column=0, row=1, sticky=(tk.N, tk.S, tk.E, tk.W))

            self.port_widgets[port_identifier] = {'frame': frame, 'status_label': status_label, 'text_widget': text_widget}
            self.port_queues[port_identifier] = queue.Queue()

    def start_experiment(self):
        # Experiment folder and dynamic handling of devices
        current_time = datetime.datetime.now().strftime("%Y_%m_%d_%H_%M_%S")
        experiment_name = self.experiment_name.get().lower().strip()
        experiment_folder = f"./{experiment_name}_{current_time}"
        os.makedirs(experiment_folder, exist_ok=True)
        self.experiment_folder = experiment_folder
        self.data_to_save = {port_identifier: [] for port_identifier in self.port_widgets.keys()}

        # Mode-specific behavior
        if self.mode.get() == "RTFED":
            # Handle RTFED mode
            self.setup_google_sheets()
        elif self.mode.get() == "TTL":
            # Handle TTL mode
            self.setup_ttl_mode()

    def setup_google_sheets(self):
        # Set up Google Sheets integration
        creds = Credentials.from_service_account_file(self.json_file_path.get(), scopes=SCOPE)
        client = gspread.authorize(creds)
        self.spreadsheet = client.open_by_key(self.spreadsheet_id.get())
        print(f"Google Sheets: Logging data to {self.spreadsheet_id.get()}")

    def setup_ttl_mode(self):
        # Handle TTL communication with GPIO setup
        pass

    def stop_experiment(self):
        stop_event.set()
        GPIO.cleanup()
        self.root.quit()

    def get_device_mappings_by_usb_port(self):
        # Dynamic detection of connected devices
        device_mappings = []
        usb_port_mapping = {'usb-0:1.1': 'Port 1', 'usb-0:1.2': 'Port 2', 'usb-0:1.3': 'Port 3', 'usb-0:1.4': 'Port 4'}
        for symlink in os.listdir('/dev/serial/by-path/'):
            symlink_path = os.path.join('/dev/serial/by-path/', symlink)
            serial_port = os.path.realpath(symlink_path)
            if 'ttyACM' in serial_port or 'ttyUSB' in serial_port:
                usb_port_path = self.get_usb_port_path_from_symlink(symlink)
                port_identifier = usb_port_mapping.get(usb_port_path)
                if port_identifier:
                    device_mappings.append({'serial_port': serial_port, 'port_identifier': port_identifier})
        return device_mappings

    def get_usb_port_path_from_symlink(self, symlink):
        match = re.search(r'usb-\d+:\d+(\.\d+)*', symlink)
        return match.group() if match else None

# Main execution
if __name__ == "__main__":
    root = tk.Tk()
    app = FED3MonitorApp(root)
    root.mainloop()
